# Reset properties in NetCDF mapfile
* Velocities
* Concentrations
* Turbulent energy and dissipation
* Vertical and horizontal eddy viscosity and diffusivity
* Transport layer composition

In [1]:
import xarray as xr
from JulesD3D.processNetCDF import fixMeshGrid, addUnderlayerCoords, makeVelocity
import numpy as np
from IPython.display import Markdown as md

In [2]:
filename = '/Users/julesblom/ThesisResults/Slope1.25/trim-36km_200m_W60ChannelRun35_compressed.nc' # Puts all of DP_BEDLYR at 0.5 m?, which is the height of the transport layer

trim = xr.open_dataset(filename)

In [3]:
new_write_filename = filename[0:-14] + '.nc'
md(f"# {new_write_filename} \n ## Double check it")

# /Users/julesblom/ThesisResults/Slope1.25/trim-36km_200m_W60ChannelRun35.nc 
 ## Double check it

In [4]:
sand = 0 
silt = 1
last_timestep = -1

# this has happened
if silt == sand:
    raise Exception("You stupid")

In [5]:
interfaces_shape = trim.W.isel(time=last_timestep).shape
centers_shape = trim.V1.isel(time=last_timestep).shape

In [6]:
all_ones_centers = np.ones(centers_shape)

all_zeros_centers = np.zeros(centers_shape)
all_zeros_interfaces = np.zeros(interfaces_shape)

## Reset velocity

In [7]:
trim['V1'][last_timestep] = all_zeros_centers      # Horizontal velocity U component
trim['U1'][last_timestep] = all_zeros_centers      # Horizontal velocity U component
trim['WPHY'][last_timestep] = all_zeros_centers    # Vertical velocity component?
trim['W'][last_timestep] = all_zeros_interfaces    # Velocity angle?

## Reset Eddies

In [8]:
trim['VICWW'][last_timestep] = all_zeros_interfaces # Vertical eddy viscosity-3D
trim['DICWW'][last_timestep] = all_zeros_interfaces # Vertical eddy diffusivity-3D
trim['VICUV'][last_timestep] = all_zeros_centers    # Horizontal eddy viscosity

In [9]:
trim.VICUV.isel(time=0).min()

<xarray.DataArray 'VICUV' ()>
array(1., dtype=float32)
Coordinates:
    time     datetime64[ns] 2020-03-01T01:00:00

## Reset Water Level

In [10]:
water_level_shape = trim.S1.isel(time=0).shape
reset_water_level = np.zeros(water_level_shape)

trim['S1'][last_timestep] = reset_water_level

## Reset concentrations

In [11]:
test_conc_shape = trim.R1.isel(time=last_timestep).shape
reset_concentrations = np.zeros(test_conc_shape)

In [12]:
trim['R1'][last_timestep] = reset_concentrations

## Reset density

In [13]:
initial_density = float(trim.RHOCONST.values)
reset_densities = all_ones_centers * initial_density

trim['RHO'][last_timestep] = reset_densities

## Reset transport layer

### Volume fraction

In [14]:
vol_frac_sand_transport_layer_end = trim.LYRFRAC.isel(time=last_timestep, LSEDTOT=sand, nlyr=0)
vol_frac_silt_transport_layer_end = trim.LYRFRAC.isel(time=last_timestep, LSEDTOT=silt, nlyr=0)

In [15]:
# Replace transport layer with 50%/50% composition
replaced_vol_frac_sand = vol_frac_sand_transport_layer_end.where(vol_frac_sand_transport_layer_end.values == 0, 0.5)
replaced_vol_frac_silt = vol_frac_silt_transport_layer_end.where(vol_frac_silt_transport_layer_end.values == 0, 0.5)

In [16]:
# Reset volume composition in transport layer at final timestep
trim.LYRFRAC[last_timestep, sand, 0] = replaced_vol_frac_sand.values
trim.LYRFRAC[last_timestep, silt, 0] = replaced_vol_frac_silt.values

### Mass of sediment

In [17]:
# for 50 cm thick transport layer!
initial_sand_mass = 400 # 16000 # [kg/m2]
initial_silt_mass = 125 # 5000  # [kg/m2]

In [18]:
mass_sand_transport_layer_end = trim.MSED.isel(time=last_timestep, LSEDTOT=sand, nlyr=0)
mass_silt_transport_layer_end = trim.MSED.isel(time=last_timestep, LSEDTOT=silt, nlyr=0)

replaced_mass_sand = mass_sand_transport_layer_end.where(mass_sand_transport_layer_end.values == 0, initial_sand_mass)
replaced_mass_silt = mass_silt_transport_layer_end.where(mass_silt_transport_layer_end.values == 0, initial_silt_mass)

In [19]:
# Reset mass of sediment in transport layer at final timestep
trim.MSED[last_timestep, sand, 0] = replaced_mass_sand.values
trim.MSED[last_timestep, silt, 0] = replaced_mass_silt.values

## Write to NetCDF
Write only last timestep to save space

In [20]:
only_last_timestep = trim.isel(time=-1)

In [21]:
only_last_timestep.load().to_netcdf(new_write_filename, mode='w', engine='netcdf4', format='NETCDF3_64BIT') 

In [22]:
trim.time.size

12

In [23]:
only_last_timestep.time.size

1